# A Star Search Experiment

## Recapping prior thoughts
The BFS algorithm I implemented in my most recent experiement was interesting for a few reasons. For one, it was able of exploring nodes in an order of lowest cost first. The algorithm was also interesting in that it allowed for costs to be associated with each action in the action space. However, there was an important limitation. Basically, the algorithm has no guidance mechanism to make educated guesses as in which general direction to expand. What this means is that the algorithm will be spending time expanding nodes that, although low in cost, are may not be nodes that will lead the drone efficiently to its destination.

## The motivation for A Star
Enter A Star Search. As I understand it, it's an alogrithm that basically allows the drone to make that educated guess using what's called a heuristic function. Like a cost value, a heuristic value is assigned to each node in the search space. When considering which node to expand to next, the algorithm will select the node with the lowest F score. In other words, the node whose sum of heuristic and cumulative cost score are lowest. Like with Uniform Cost, A Star can be implemented using a Priority Queue data structure in Python, and that's what I'll be doing in this notebook. 

In [53]:
from queue import PriorityQueue
import numpy as np
from enum import Enum

class Action(Enum):
    """
    This class defines the action set. Each action is a 3-element tuple where the first two elements describe
    the vertical and horizontal grid motions or translations, while the final element of the tuple describes
    the action's cost. 
    
    """
    LEFT = (0, -1, 1)
    RIGHT = (0, 1, 1)
    UP = (-1, 0, 1)
    DOWN = (1, 0, 1)
    NORTHEAST = (-1, 1, np.sqrt(2))
    NORTHWEST = (-1, -1, np.sqrt(2))
    SOUTHEAST = (1, 1, np.sqrt(2))
    SOUTHWEST = (1, -1, np.sqrt(2))
    
    def __str__(self):
        if self == self.LEFT:
            return '<'
        elif self == self.RIGHT:
            return '>'
        elif self == self.UP:
            return '^'
        elif self == self.DOWN:
            return 'v'
        elif self == self.NORTHEAST:
            return '*'
        elif self == self.NORTHWEST:
            return '*'
        elif self == self.SOUTHEAST:
            return '*'
        elif self == self.SOUTHWEST:
            return '*'
        
    @property
    def cost(self):
        return self.value[2]
    @property
    def delta(self):
        return (self.value[0], self.value[1])

def valid_actions(grid, current_node):
    """
    Returns a list of valid actions given a grid and given a node.
    """
    valid = [Action.UP, Action.LEFT, Action.RIGHT, Action.DOWN, Action.NORTHEAST, Action.NORTHWEST, Action.SOUTHEAST, Action.SOUTHWEST]
    n, m = grid.shape[0] - 1, grid.shape[1] - 1
    x, y = current_node
    
    # Removes actions leading to nodes that are off-grid.
    if x - 1 < 0:
        valid.remove(Action.UP)
    if x + 1 > n:
        valid.remove(Action.DOWN)
    if y - 1 < 0:
        valid.remove(Action.LEFT)
    if y + 1 > n:
        valid.remove(Action.RIGHT)
    if x - 1 < 0 and y - 1 < 0:
        valid.remove(Action.NORTHWEST)
    if x - 1 < 0 and y + 1 > m:
        valid.remove(Action.NORTHEAST)
    if x + 1 > n and y - 1 < 0:
        valid.remove(Action.SOUTHWEST)
    if x + 1 > n and y + 1 > m:
        valid.remove(Action.SOUTHEAST)
    
    # Removes actions leading to nodes that are obstacles.
    if x - 1 >= 0 and x - 1 <= n and y - 1 >= 0 and y - 1 <= m and grid[x-1, y-1] == 1:
        valid.remove(Action.NORTHWEST)
    if x - 1 >= 0 and x - 1 <= n and y + 1 >= 0 and y + 1 <= m and grid[x-1, y+1] == 1:
        valid.remove(Action.NORTHEAST)
    if x + 1 >= 0 and x + 1 <= n and y - 1 >= 0 and y - 1 <= m and grid[x+1, y-1] == 1:
        valid.remove(Action.SOUTHWEST)
    if x + 1 >= 0 and x + 1 <= n and y + 1 >= 0 and y + 1 <= m and grid[x+1, y+1] == 1:
        valid.remove(Action.SOUTHEAST)
    if x - 1 >= 0 and x - 1 <= n and y >= 0 and y <= m and grid[x-1, y] == 1:
        valid.remove(Action.UP)
    if x + 1 >= 0 and x + 1 <= n and y >= 0 and y <= m and grid[x+1, y] == 1:
        valid.remove(Action.DOWN)
    if x >= 0 and x <= n and y - 1 >= 0 and y - 1 <= m and grid[x, y-1] == 1:
        valid.remove(Action.LEFT)
    if x >= 0 and x <= n and y + 1 >= 0 and y + 1 <= m and grid[x, y+1] == 1:
        valid.remove(Action.RIGHT)
    
    return valid

def visualize_path(grid, path, start):
    sgrid = np.zeros(np.shape(grid), dtype=str)
    sgrid[:] = ' '
    sgrid[grid[:] == 1] = 'O'
    
    pos = start
    for a in path:
        da = a.value
        sgrid[pos[0] + da[0], pos[1] + da[1]] = str(a)
        pos = (pos[0] + da[0], pos[1] + da[1])
    sgrid[pos[0], pos[1]] = 'G'
    sgrid[start[0], start[1]] = 'S'
    
    return sgrid

def heuristic(position, goal_position):
    """
    Computes and returns the straight-line distance between the a given node and a goal node.
    """
    h = np.sqrt((position[0] - goal_position[0])**2 + (position[1] - goal_position[1])**2) 
    return h

def a_star(grid, h, start, goal):
    """
    Executes A Star Search given a grid, heuristic array, a start goal, and an end goal. 
    Returns the path if one is found. 
    """
    
    # First, initializes data structures.
    path = []
    path_cost = 0
    queue = PriorityQueue()
    queue.put((0, start))
    visited = set(start)
    branch = {}
    found = False
    
    # Then, begins the A Star. The algorithm will expand the lowest queue-cost node, where queue-cost
    # is the sum of the heuristic cost and the path cost. 
    while not queue.empty():
        item = queue.get()
        current_node = item[1]
        
        if current_node == start:
            current_cost = 0.0
        else:
            current_cost = branch[current_node][0]
        
        if current_node == goal:
            print('Found a path.')
            found = True
            break
        else:
            for action in valid_actions(grid, current_node):
                da = action.delta
                next_node = (current_node[0] + da[0], current_node[1] + da[1])
                branch_cost = current_cost + action.cost
                queue_cost = branch_cost + h(next_node, goal)
                
                if next_node not in visited:
                    visited.add(next_node)
                    branch[next_node] = (branch_cost, current_node, action)
                    queue.put((queue_cost, next_node))
    
    # After, retraces steps, returning the step-by-step actions and total path-cost required to get there.
    if found:
        n = goal
        path_cost = branch[n][0]
        while branch[n][1] != start:
            path.append(branch[n][2])
            n = branch[n][1]
        path.append(branch[n][2])
    else:
        print('************')
        print('Failed to find a path.')
        print('************')
        
    return path[::-1], path_cost

    
    

In [54]:
start = (0, 0)
goal = (4, 4)

grid = np.array([
    [0, 1, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0],
    [0, 1, 0, 0, 0, 0],
    [0, 0, 0, 1, 1, 0],
    [0, 0, 0, 1, 0, 0],
])

In [55]:
path, cost = a_star(grid, heuristic, start, goal)
print(path, cost)

Found a path.
[<Action.SOUTHEAST: (1, 1, 1.4142135623730951)>, <Action.SOUTHEAST: (1, 1, 1.4142135623730951)>, <Action.RIGHT: (0, 1, 1)>, <Action.RIGHT: (0, 1, 1)>, <Action.SOUTHEAST: (1, 1, 1.4142135623730951)>, <Action.SOUTHWEST: (1, -1, 1.4142135623730951)>] 7.65685424949238


In [56]:
visualize_path(grid, path, start)

array([['S', 'O', ' ', ' ', ' ', ' '],
       [' ', '*', ' ', ' ', ' ', ' '],
       [' ', 'O', '*', '>', '>', ' '],
       [' ', ' ', ' ', 'O', 'O', '*'],
       [' ', ' ', ' ', 'O', 'G', ' ']], dtype='<U1')